In [2]:
import requests
import os
from pprint import pprint
import json
import re
import re
import nltk
from stop_words import get_stop_words
from nltk.corpus import stopwords

In [ ]:
apikey = 'Your_API_Key'
query_url = f"https://api.nytimes.com/svc/archive/v1/2020/6.json?api-key={apikey}"
r = requests.get(query_url)
data = r.json()

In [ ]:
stop_words = list(get_stop_words('en'))         
nltk_words = list(stopwords.words('english'))   
stop_words.extend(nltk_words)


years = ['2020', '2021']
keyword_dict = {}
months = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']


for year in years:
    for month in months:
        query_url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apikey}"
        r = requests.get(query_url)
        data = r.json()
        articles = data["response"]["docs"]
        keyword_dict["year{y}month{m}".format(y=year, m=month)] = [set() for index in range(31)]
        for article in articles:
            if article['section_name'] == 'Business Day':
                corresponding_set = int(article['pub_date'][8:10])-1
                for word in article['keywords']:
                    re_word = re.sub('[^a-zA-Z0-9-. ]', '', word['value'].lower())
                    tokens = nltk.word_tokenize(re_word)
                    for token in tokens:
                        if not token in stop_words:
                            keyword_dict["{y}-{m}-{d}".format(y=year, m=month,d=str(corresponding_set+1))][corresponding_set].add(re.sub('\-$', '', token))
            
            else:
                continue

In [ ]:
with open('NYT2020_2021_keywords.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

------
<h1>Create new variables</h1>

In [20]:
# create the t3_df
import pandas as pd

fp = '/Users/victor/Desktop/CS/UST_summer/t3_df.csv'
sc = ['id','author','created_utc','title']
t3_df = pd.read_csv(fp, low_memory = False)

In [21]:
t3_df.head()

,Unnamed: 0,id,author,title,is_bot
0,0,mhj2yd,USWD_Bank,NFT Collectable: USWD (United States Web Dollar),False
1,2,mhj3ew,KingMacias1,We made it to Vice News you dirty apes! Any pu...,False
2,3,mhj3oa,jt989898,Polygon?,False
3,4,mhj3vq,KenAdamsonPrime,"Ubiquiti ($UI) gets PWND by hackers, stock goe...",False
4,6,mhj4o5,lexmark0331,HOFV mega squeeze,False


In [ ]:
from datetime import datetime, timedelta
import numpy as np

def generate_NYT_keyword(t3_df, keyword_dict):
    has_key = []
    for index, row in t3_df.iterrows():
        dt = datetime.fromtimestamp(row['created_utc'])
        prev_dt = dt - timedelta(days=1)
        # datetime obj to string
        dt = str(dt).split('')[0]
        prev_dt = str(prev_dt).split('')[0]
        
        # tokenise the title
        title = row['title']
        if title == ' ':
            has_key.append(np.nan)
            continue
        
        title_tokens = set(nltk.word_tokenize(row['title']))
        
        keys = keyword_dict[dt] # it returns a set of keywords from this date.
        try:
            prev_key = keyword_dict[prev_dt] # it returns the keyword from the previous day
            keys = keys.union(prev_key)
        except:
            print('it is the start day of the dataframe')
        
        # check the set intersection
        result = keys.intersection(title_tokens)
        
        if len(result) != 0:
            has_key.append(1)
        else: has_key.append(0)
    
    return has_key
